In [1]:
#!pip install rdflib
import rdflib
from rdflib import Graph
import pandas as pd
import json
import urllib.parse

In [2]:
g = Graph()
g.parse("kg-mic.ttl")

<Graph identifier=Naa1cef661b914ad49718bd46e2c01577 (<class 'rdflib.graph.Graph'>)>

In [3]:
#g.serialize(destination="kg-mic.ttl")

<Graph identifier=N39181cd92372482691fdbadb96595a2b (<class 'rdflib.graph.Graph'>)>

In [3]:
print(len(g))

6827261


In [4]:
df = pd.read_csv("C:/Users/Dominika/Documents/SKOLA/ING/DIPLOMKA/output/test_data.tsv", delimiter = "\t")
df

,subject,CHEBI:100147,CHEBI:12931,CHEBI:12936,CHEBI:132943,CHEBI:132950,CHEBI:133748,CHEBI:13705,CHEBI:143136,CHEBI:14314,...,temp_range:very_low,temperature:mesophilic,temperature:psychrophilic,temperature:thermophilic,trophic_type:chemoheterotrophy,trophic_type:chemoorganoheterotrophy,trophic_type:chemoorganotrophy,trophic_type:chemotrophy,trophic_type:heterotrophy,trophic_type:organotrophy
0,NCBITaxon:1122600,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,NCBITaxon:684552,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,NCBITaxon:452922,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,NCBITaxon:447421,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,NCBITaxon:273384,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,NCBITaxon:1465756,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1227,NCBITaxon:872983,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1228,NCBITaxon:184924,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1229,NCBITaxon:1223526,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [5]:
taxons = []
for i in range(len(df)): 
    taxons.append(df.iloc[i, 0])

In [6]:
len(taxons)

1231

In [7]:

bioregistry_prefixes = {}
f = open("C:/Users/Dominika/Documents/SKOLA/ING/DIPLOMKA/registry.json")
data = json.load(f)
for entity in data.values():
    bioregistry_prefixes[entity["prefix"]] = {"name": entity["name"], "uri_format": entity["uri_format"]}
f.close()
del data



def extract_uri(id: str):
    uri_parts = id.split(":", 1)
    if len(uri_parts) == 1:
        encoded_id = urllib.parse.quote(id, safe="")
        return f"urn:unknown:{encoded_id}"
    elif len(uri_parts) > 1:
        prefix = uri_parts[0].lower()
        def_prefix = bioregistry_prefixes.get(prefix)
        if def_prefix is None:
            encoded_id = urllib.parse.quote(id, safe="")
            return f"urn:unknown:{encoded_id}"
        else:
            uf = def_prefix["uri_format"]  # type: str
            if uf is None or "$1" not in uf:
                return f"https://bioregistry.io/{prefix}:{uri_parts[1]}"
            else:
                return uf.replace("$1", uri_parts[1])
    return None


taxons_prefixed = []
for row in taxons:
    uri_1 = extract_uri(row)
    taxons_prefixed.append(uri_1)

len(taxons_prefixed)

1231

In [8]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""
new_graph = Graph()

for row in taxons_prefixed:
    subjectitos = rdflib.term.URIRef(row)
    filtered_gq = g.query(query, initBindings = {
        "s": subjectitos
    })
    new_graph += filtered_gq

print(len(new_graph))



39049


In [ ]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""

for row in taxons_prefixed:
    objectitos = rdflib.term.URIRef(row)
    filtered_gq = g.query(query, initBindings = {
        "o": objectitos
    })
    new_graph += filtered_gq

print(len(new_graph))

In [9]:
new_real_graph = Graph()
for row in new_graph:
    new_real_graph.add(row)

len(new_real_graph)

39049

In [10]:
for row in new_real_graph:
    g.remove(row)

In [11]:
len(g)

6788212

In [12]:
g.serialize(destination="train_data_from_spaqrl_onlysub.ttl")

<Graph identifier=Naa1cef661b914ad49718bd46e2c01577 (<class 'rdflib.graph.Graph'>)>

In [2]:
g = Graph()

g.parse("train_data_from_spaqrl_onlysub.ttl")

KeyboardInterrupt: 

In [5]:
q = """
SELECT ?s ?p ?o
{
    ?s ?p ?o
}
"""

gq = g.query(q)


train_taxons = []

for row in gq:
    if (row.s[0:40] == "http://purl.obolibrary.org/obo/NCBITaxon"):
        train_taxons.append(rdflib.term.URIRef(row.s))

print(len(train_taxons))

3783982


In [6]:
import csv
with open('train_taxonst_second_try.tsv', 'w', newline='\n') as f:
     
    # using csv.writer method from CSV package
    write = csv.writer(f)
     
    write.writerows([train_taxons])

In [3]:
df = pd.read_csv("train_taxonst_second_try.tsv", lineterminator = ",", header=None)


In [4]:
df

,0
0,http://purl.obolibrary.org/obo/NCBITaxon_1241306
1,http://purl.obolibrary.org/obo/NCBITaxon_2719337
2,http://purl.obolibrary.org/obo/NCBITaxon_1004261
3,http://purl.obolibrary.org/obo/NCBITaxon_1633686
4,http://purl.obolibrary.org/obo/NCBITaxon_2644557
...,...
3783977,http://purl.obolibrary.org/obo/NCBITaxon_1051666
3783978,http://purl.obolibrary.org/obo/NCBITaxon_1250044
3783979,http://purl.obolibrary.org/obo/NCBITaxon_1467204
3783980,http://purl.obolibrary.org/obo/NCBITaxon_2804029


In [5]:
new_g = Graph()

new_g.parse("kg-mic.ttl")

<Graph identifier=Nfcf0a50203ae47ff82aa712fa6ce8a8c (<class 'rdflib.graph.Graph'>)>

In [6]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""
print("zacatek: ",len(new_g))

for i in range(len(df)): 
    subjectitos = rdflib.term.URIRef(df[0][i])
    filtered_gq = new_g.query(query, initBindings = {
        "s": subjectitos
    })
    for row in filtered_gq:
        new_g.remove(row)
    if i % 100000 == 0:
        print(i)

print("konec: ",len(new_g))

zacatek:  6827261
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
konec:  3043279


In [7]:
new_g.serialize(destination="test_data_from_spaqrl_onlysub.ttl")

<Graph identifier=Nfcf0a50203ae47ff82aa712fa6ce8a8c (<class 'rdflib.graph.Graph'>)>

In [44]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""
new_graph_train = Graph()

filtered_gq = g.query(query, initBindings = {
        "o": rdflib.term.URIRef("urn:unknown:medium%3A92")
    })

print(len(filtered_gq))

157


In [45]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""

#new_graph_show = Graph()
for row in filtered_gq:
    filtered = g.query(query, initBindings = {
        "s": rdflib.term.URIRef(row.s)
    })
    new_graph_show += filtered

print(len(new_graph_show))

42839


In [46]:
query = """
SELECT ?s ?p ?o
WHERE {
  ?s ?p ?o .
}
"""

for row in filtered_gq:
    filtered = g.query(query, initBindings = {
        "o": rdflib.term.URIRef(row.s)
    })
    new_graph_show += filtered

print(len(new_graph_show))

44917


In [47]:
show_graph = Graph()
for row in new_graph_show:
    show_graph.add(row)

len(show_graph)

44917

In [48]:
show_graph.serialize(destination="bestof_graph.ttl")

<Graph identifier=N2e6f0ecb84e54dcf82d2825c5dd28301 (<class 'rdflib.graph.Graph'>)>